In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
# read in all the words
words = open('names.txt', 'r').read().splitlines()
words[:3]

['emma', 'olivia', 'ava']

In [4]:
#build the vocab of chars and mappings to integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi["."] = 0
itos = {i:s for s, i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [6]:
# build the dataset
block_size = 3
def build_dataset(words):
    X, Y = [], []

    for w in words:
        context = [0] * block_size
        for ch in w + ".":
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])
        

torch.Size([182580, 3]) torch.Size([182580])
torch.Size([22767, 3]) torch.Size([22767])
torch.Size([22799, 3]) torch.Size([22799])


In [ ]:

class Linear:

    def __init__(self, fan_in, fan_out, bias=True):
        self.weight = torch.randn((fan_in, fan_out), generator=g) / fan_in**0.5
        self.bias = torch.zeros(fan_out) if bias else None
    
    def __call__(self, x):
        self.out = x @ self.weight
        if self.bias is not None:
            self.out += self.bias
        return self.out

    def parameters(self):
        return [self.weight] + ([] if self.bias is None else [self.bias])

class BatchNorm1d:

    def __init__(self, dim, eps=1e-5, momentum=0.1):
        self.eps = eps
        self.momentum = momentum
        self.training = True
        #parameters (trained with backprop)
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)
        #buffers (trained with a running 'momentum update')
        self.running_mean = torch.zeros(dim)
        self.running_var = torch.ones(dim)
    
    def __call__(self, x):
        # calculate the forward pass
        if self.training:
            xmean = x.mean(0, keepdim=True)
            xvar = x.var(0, keepdim=True, unbiased=True)
        else:
            xmean = self.running_mean
            xvar = self.running_var
        xhat = [x - xmean] / torch.sqrt(xvar + self.eps)
        self.out = self.gamma * xhat + self.beta
        # update the buffers
        if self.training:
            with torch.no_grad():
                self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * xmean
                self.running_var = (1 - self.momentum) * self.running_var + self.momentum * xvar
        return self.out
    
    def parameters(self):
        return [self.gamma, self.beta] 

class Tanh:

    def __call__(self, x):
        self.out = torch.tanh(x)
        return self.out
    
    def parameters(self):
        return []

n_embd = 10  # the dimensionality of the character embedding vectors
n_hidden = 200  #hidden layer
g  = torch.Generator().manual_seed(2147729)
C  = torch.randn((vocab_size, n_embd),              generator=g)
layers = [
    Linerar(n_embd * block_size, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
    Linerar(           n_hidden, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
    Linerar(           n_hidden, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
    Linerar(           n_hidden, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
    Linerar(           n_hidden, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
    Linerar(           n_hidden, vocab_size, bias=False), BatchNorm1d(vocab_size), 
]

with torch.o_grad():
    # last layer: make less confident
    layers[-1].gamma *= 0.1
    for layer in layers[:-1]:
        if isinstance(layer, Liner):
            layer.weight *= 1.0

parameters = [C] + [p for layer in layers for p in layer.parameters()]
print(sum(p.nelement() for p in parameters))
for p in parameters:
    p.requires_grad = True


In [ ]:
# Optimazation
max_steps = 200000
batch_size = 32
lossi = []

for i in range(max_steps):

    #minibatch construct
    ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)  #random indices
    Xb, Yb = Xtr[ix], Ytr[ix]

    #forward pass -----------
    emb = C[Xb]  #embed characters into vectors
    embcat = emb.view(emb.shape[0], -1)  #concatanate the vectors or flatten

    for layer in layers:
        x = layer(x)
    loss = F.cross_entropy(x, Yb)


    #backward pass ---------------
    for layer in layers:
        layer.out.retain_grad()
    for p in parameters:
        p.grad = None
    loss.backward()

    # update
    lr = 0.1 if i < 100000 else 0.01
    for p in parameters:
        p.data += -lr * p.grad

    # track stats
    if i % 10000 == 0:  # print every once in a while
        print(f"{i:7d}/{max_steps:7d}: {loss.item():.4f} :lr={lr}")
    lossi.append(loss.log10().item())

    with torch.no_grad():
        ud.append([((lr*p.grad).std() / p.data.std()).log10().item() for p in parameters])

    # break
    if i > 1000:
        break






      0/ 200000: 3.3184 :lr=0.1
  10000/ 200000: 2.3204 :lr=0.1
  20000/ 200000: 2.5286 :lr=0.1
  30000/ 200000: 2.0454 :lr=0.1
  40000/ 200000: 2.0777 :lr=0.1
  50000/ 200000: 2.1664 :lr=0.1
  60000/ 200000: 2.1039 :lr=0.1
  70000/ 200000: 2.3111 :lr=0.1
  80000/ 200000: 1.9221 :lr=0.1
  90000/ 200000: 2.4140 :lr=0.1
 100000/ 200000: 2.2978 :lr=0.01
 110000/ 200000: 2.1148 :lr=0.01
 120000/ 200000: 2.3658 :lr=0.01
 130000/ 200000: 2.0686 :lr=0.01
 140000/ 200000: 2.1893 :lr=0.01
 150000/ 200000: 2.0851 :lr=0.01
 160000/ 200000: 1.8014 :lr=0.01
 170000/ 200000: 2.1003 :lr=0.01
 180000/ 200000: 1.8534 :lr=0.01
 190000/ 200000: 2.0822 :lr=0.01


In [ ]:
plt.figure(figsize=(20, 4))
legends = []
for i, layer in enumerate(layers[:-1])
if isinstance(layer, Tanh):
    t = layer.out
    print('layer %d (%10s): mean %+.2f, std %.2f, satured: %2.f%%' % (i, layer.__class__.__name__, t.eman(), t.std(), (t.abs() > 0.97).float().mean()*100))
    hy, hx = torch.histogram(t, density=True)
    plt.plot(hx[:-1], detach(), hy.detach())
    legends.append(f'layer {i} ({layer.__class__.__name__})')
plt.legend(legends)
plt.title('activation distribution')